In [ ]:
dataset = r"C:\Users\Lucia\Desktop\PROYECTO_MLP_PABLO_XAVIER\PROYECTO_MLP_PABLO_XAVIER_MENDEZ\src\data\process\telecom.csv"
df = pd.read_csv(dataset, parse_dates=['DATE'])

import pandas as pd

calls_per_date_df = pd.DataFrame(calls_per_date, columns=["CALLS"])

calls_per_date_df.index = pd.to_datetime(calls_per_date_df.index).to_period('D')

print(calls_per_date_df)

moving_average = calls_per_date_df.rolling(
    window=90,       
    center=True,      
    min_periods=45,  
).mean()              

ax = calls_per_date_df.plot(style=".", color="0.5")
moving_average.plot(
    ax=ax, linewidth=3, title="llamadas diarias - 90 -Day Moving Average", legend=False,)

from statsmodels.tsa.deterministic import DeterministicProcess

dp = DeterministicProcess(
    index=calls_per_date_df.index,  
    constant=True,       # característica dummy para el sesgo (y_intercept)
    order=1,             # la variable ficticia del tiempo (tendencia)
    drop=True,           # elimine términos si es necesario para evitar colinealidad
)
# `in_sample` crea características para las fechas dadas en el argumento `index
X = dp.in_sample()

X.head()

from sklearn.linear_model import LinearRegression

y = calls_per_date_df["CALLS"]

# La intercepción es el mismo que el `const` característica de
# DeterministicProcess. LinearRegression se comporta mal con 
# características duplicadas, por lo que necesitamos estar seguros de excluirlos.
model = LinearRegression(fit_intercept=False)
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)

ax = calls_per_date_df.plot(style=".", color="0.5", title="llamadas diarias - Linear Trend")
_ = y_pred.plot(ax=ax, linewidth=3, label="Trend")

X = dp.out_of_sample(steps=30)

y_fore = pd.Series(model.predict(X), index=X.index)

y_fore.head()

calls_april = calls_per_date_df["2021-03":]
y_pred_april = y_pred["2021-03":]
y_fore_april = y_fore["2021-03":]

# Configurar parámetros de gráfico
plot_params = {
    "kind": "line",
    "title": "Llamadas diarias - Pronóstico de tendencia lineal"
}

# Crear gráfico
ax = calls_april.plot(**plot_params)
ax = y_pred_april.plot(ax=ax, linewidth=3, label="Tendencia")
ax = y_fore_april.plot(ax=ax, linewidth=3, label="Pronóstico de tendencia", color="C3")

# Mostrar leyenda
ax.legend()

# Mostrar el gráfico
plt.show()

mse = mean_squared_error(calls_per_date_df['CALLS'], y_pred)
print("Mean Squared Error (MSE):", mse)

from sklearn.metrics import mean_absolute_error


mae = mean_absolute_error(calls_per_date_df['CALLS'], y_pred)
print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import r2_score

# y_true son los valores reales y y_pred son las predicciones del modelo
r2 = r2_score(calls_per_date_df['CALLS'], y_pred)

print("R cuadrado (R^2):", r2)